In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [20]:
learning_rate = 0.01
training_epochs = 1
eps = 0.007


# Single image classification
x = tf.placeholder(tf.float32, (None, 784)) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

grad_input = tf.gradients(cost, x)[0]
init = tf.global_variables_initializer()
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    #Image to train
    dummy = mnist.train.images[0]
    dummy = np.reshape(dummy,(1,784))
    dummy_l = mnist.train.labels[0]
    dummy_l = dummy_l[np.newaxis,:]
    
#     print(dummy_l.shape)
    
    #Image to test
    dummy2 = mnist.train.images[0]
    dummy2 = dummy2[np.newaxis,:]
    dummy2_l = mnist.train.labels[0]
    dummy2_l = dummy2_l[np.newaxis,:]
        
    # Run optimization op (backprop) and cost op (to get loss value)
    _, c = sess.run([optimizer, cost], feed_dict={x: dummy,
                                                          y: dummy_l})
        
    print("Epoch:", '%04d' % (1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    grad = sess.run([grad_input], feed_dict={x: dummy,y: dummy_l})
    
    print(len(grad))
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: dummy2, y: dummy2_l}))
    
    img_ad = dummy + eps * np.sign(grad[mnist.train.labels.tolist()[0].index(1.0)])
    
    print("Accuracy:", accuracy.eval({x: img_ad, y: dummy2_l}))
#this will get u the gradients of the test images.
#Then you have to create adversial images using this gradient
# for i in range(100):
#     img_ad = mnist.test.images[i] + eps * np.sign(grad[mnist.test.labels.tolist()[i].index(1.0)])
# adversarial_images.append(img_ad)

Epoch: 0001 cost= 2.302585125
Optimization Finished!
1
Accuracy: 1.0


IndexError: list index out of range

In [14]:
print(mnist.train.labels.tolist()[0].index(1.0))

7


In [ ]:
#Hyperparameters
learning_rate = 0.01
training_epochs = 1


# Single image classification
x = tf.placeholder(tf.float32, (None, 784)) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes
x_image = tf.reshape(x, [-1,28,28,1]) # mnist image comes in as 784 vector


# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
keep_prob = tf.placeholder(tf.float32)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

deriv = tf.gradients(cost, x)[0]
image_adv = tf.stop_gradient(x - tf.multiply(2.5,tf.sign(deriv)*learning_rate))
image_adv = tf.clip_by_value(image_adv, 0, 1)


# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    #Image to train
    dummy = mnist.train.images[0]
    dummy = np.reshape(dummy,(1,784))
    dummy_l = mnist.train.labels[0]
    dummy_l = dummy_l[np.newaxis,:]
    
    #Image to test
    dummy2 = mnist.train.images[0]
    dummy2 = dummy2[np.newaxis,:]
    dummy2_l = mnist.train.labels[0]
    dummy2_l = dummy2_l[np.newaxis,:]
        
    # Run optimization op (backprop) and cost op (to get loss value)
    _, c = sess.run([optimizer, cost], feed_dict={x: dummy,
                                                          y: dummy_l})
        
    print("Epoch:", '%04d' % (1), "cost=", "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
#     print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
    print("Accuracy:", accuracy.eval({x: dummy2, y: dummy2_l}))
    
    #Computes gradients, I think or hope!
    dydx = sess.run(deriv,  feed_dict = {x: dummy, y:dummy_l}) # can't seem to access 'deriv' w/o running this
#     _ = sess.run([grad_input, cost], feed_dict={x: dummy,
#                                                           y: dummy_l})
    x_adv = sess.run(image_adv, {x: dummy2, y:dummy_l})
    x_adv = x_adv.reshape(1,784)
    print("Accuracy:", accuracy.eval({x: x_adv, y: dummy_l}))
    
    plt.imshow(x_adv.reshape((28,28)), cmap='Greys')
    

In [ ]:
#LOGISTIC REGRESSION

# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    print("Accuracy:", accuracy.eval({x: dummy, y: dummy_l}))

#https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/logistic_regression.py

In [ ]:
#https://github.com/jasonicarter/MNIST-adversarial-images/blob/master/MNIST-adversarial-images.ipynb
image_list = mnist.train.images[0:9]
image_list_labels = mnist.train.labels[0:9]

# https://matplotlib.org/mpl_toolkits/axes_grid/users/overview.html#imagegrid
fig = plt.figure(1, (5., 5.))
grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(3, 3),  # creates 2x2 grid of axes
                 axes_pad=0.3,  # pad between axes in inch.
                 )

for i in range(len(image_list)):
    image = image_list[i].reshape(28,28)
    grid[i].imshow(image)
    grid[i].set_title('Label: {0}'.format(image_list_labels[i].argmax()))

plt.show()